# Kryptos k4

This notebook demonstrates how K4 is most likely going to be broken.

This is based on a number of observations and experiments which have taken place over the 6 years between my first discovering the cipher in 2014, and today, June 2020.

To execute this notebook, you will need:

- Python 3.7 or later
- pandas dataframe library
- Jupyter
  - ipywidgets [https://ipywidgets.readthedocs.io/en/latest/](https://ipywidgets.readthedocs.io/en/latest/)
  - ipyevents [https://github.com/mwcraig/ipyevents](https://github.com/mwcraig/ipyevents)


## Deciphering

The principle of K4 is fairly simple.

### The grid
- Take the ciphertext, then subtract each character from Z. I call this 'lacuna' because 'lacuna' means a missing piece of the puzzle.
  - The word is derived from the etymology of the word `LAQUEO` which is formed from the misspellings and their associated counterparts
- take the lacuna and subtract this from the ciphertext
- take the result and subtract this from the lacuna
- repeat until you have a 97x26 table
- split the table into two
  - table 1 is all rows which are fully even in nature
  - table 2 is all rows which contain a mixture of odd and even numbers
    - there are **no** rows which are all odd
  - for the even table, add a row of Z in each column
- for the table containing mixed rows:
  - the header of the table is the alphabet
  - The footer of the table is the alphabet reversed
  - The left index is derived from the even table
  - the right index is derived by looping the alphabet on 13
- for the even table
  - The header of the table is derived from the duplicated columns
    - create a table from the ciphertext, remove all duplicated characters then merge the column headers into a single index where column values are still duplicated
    - can also be derived from x + 13
  - For the footer of the table and the left side of the table:
    - match the index values from the header
  - for the right side:
    - match the index values to the values in the column at position 12
- plot a grid on the table matching the same characters as the ciphertext
  - If K is found, to form the top left, use V on every alternate iteration of K else use K
  - If M is found, use K every iteration
  - if V is found, use K on every alternate iteration else use V
  - If Z is found, use V on every iteration
  
### Deciphering the grid
- The character is in one of the squares but is found by a set of rules which seem to be determined if:
  - Neither the original cipher character, nor the ciphers lacuna is in the table, start at top left
  - If the ciphers index value is even, go to top left
  - If the ciphers lacuna value is in the tables, invert the location
  - if the cipher character is in the table, invert the inversion
- Once the character has been found, it needs to be placed into one of 4 ciphers

I have not yet discovered how this is achieved.
  
M and Z may alternate K and V. We cannot be certain until the bulk of the cipher is unpicked.

## The machine

The following classes define the machine with the tables generated at the bottom of this notebook.

I've used a class based system as eventually I'll tie this into [PyKryptos](https://github.com/mproffitt/pykryptos) and I find it easier to read code structured in this manner.

> Be warned
> This code represents a first draft. It's been in no way prettified or simplified. These are normally tasks I undertake once I've a firm grasp of the algorithm being developed. If you can't understand something, ask.

### imports
For the purposes of this cipher, these are all the imports you require.

```
pip install --upgrade pandas ipython widgetsnbextension ipyevents jupyter
```

In [1]:
import pandas as pd
from itertools import combinations, permutations, product
from IPython import display
from time import sleep

from ipywidgets import Label, HTML, VBox, HBox, Output
from ipyevents import Event
pd.set_option('display.max_columns', None)

### ciphertext
We define the ciphertext at the top of this notebook so those using it can tell without reading the whole file what is being deciphered.

In [2]:
ciphertext = 'OBKRUOXOGHULBSOLIFBBWFLRVQQPRNGKSSOTWTQSJQSSEKZZWATJKLUDIAWINFBNYPVTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR'

### Helpers class
In order to support the cipher, we need to define a number of shared methods and attributes used throughout the different components.

These methods are for basic operations such as converting between an alphabetic character and a numerical index (`a2i`) as well as the inverse.

We also include more advanced cipher methods here such as the distance calculator to creating the initial table.

In [3]:
class Helpers(object):
    """
    The helpers class is actually the core of this cipher, defining the methods used to create the
    tables, handle the conversion to/from ciphertext and switch between alphabetic and numerical systems.
    """
    alphabet = [
        'A', 'B', 'C', 'D', 'E', 'F', 'G',
        'H', 'I', 'J', 'K', 'L', 'M', 'N',
        'O', 'P', 'Q', 'R', 'S', 'T', 'U',
        'V', 'W', 'X', 'Y', 'Z'
    ]
    
    cache = {
        'calculator': None
    }
    cache_locked = False
    
    def a2i(self, ch):
        """
        Converts an alphabetic character into a numerical index
        
        :param: char c
        """
        return self.alphabet.index(ch.upper()) + 1

    def i2a(self, i):
        """
        Converts a numerical index into a alphabetic character
        
        :param: int i
        """
        return self.alphabet[(i-1)]
    
    def distanceto(self, x, y):
        """
        Addition vector moving through Z

        :param: char x The starting character
        :param: char y The character to calculate the distance to

        :return char
        """
        a = self.a2i(x)
        b = self.a2i(y)
        c = (26 - a) + b
        return self.i2a(c % 26)

    def distancefrom(self, x, y):
        """
        Subtraction vector moving through Z

        :param: char x The starting character
        :param: char y The character to calculate the distance from

        :return char
        """
        a = self.a2i(x)
        b = self.a2i(y)

        c = ((26 - a) - b)
        return self.i2a(c % 26)
    
    def polarity(self, message):
        """
        Test whether the characters in the current string are all odd, even or a mixture
        
        :param: string message
        
        :return: char
        
        This method will return one of
        
        - O if all characters are odd
        - E if all characters are even
        - M if the characters are a mix of odd and even
        """
        return (
            'E' if all(j % 2 == 0 for j in [self.a2i(t) for t in message])
            else 'O' if all(not (j % 2 == 0) for j in [self.a2i(t) for t in message])
            else 'M'
        )

    def distance_calculator(self, start, end):
        """
        Calculates a table of distances between the start and end positions
        
        :param: string start This is the alphabet to start the calculation from
        :param: string end This represents the 'other' end of the alphabet.
        
        :return: list
        
        Usually the end of the alphabet is represented as 
        `distancefrom(c, Z) for c in ciphertext`
        
        This will return a 97x26 grid of all possible positions.
        Because it takes so long to build, the result of this is
        stored in memory for re-use throughout the cipher.
        """
        while self.cache_locked:
            sleep(0.1)
    
        if not self.cache['calculator']:
            self.cache_locked = True
            distances = [
                (start, self.polarity(start)),
                (end, self.polarity(end)),
            ]
            completed = []
            for pos in range(26):
                for item in combinations([d[0] for d in distances], 2):
                    if item in completed:
                        continue
                    c = item[0]
                    s = item[1]
                    for i in range(6):
                        z = ''
                        for j in range(len(c)):
                            z += self.distanceto(c[j], s[j])

                        e = self.polarity(z)
                        if (z, e) not in distances:
                            distances.append((z, e))
                            p = ' '.join([str(self.a2i(t)).zfill(2) for t in z])
                        c = s
                        s = z

                    completed.append(item)
                pos += 1

            self.cache['calculator'] = distances
            self.cache_locked = False
        return self.cache['calculator']

helpers = Helpers()

In [4]:
class Highlighter(object):
    """
    Applies grid formatting to a pandas dataframe object
    
    This is a helper class which draws up the grid. 
    """
    _df         = None
    _applied    = None
    _grid       = None
    _active_pos = None
    _cipher_pos = None
    
    def __init__(self, df, grid):
        self._df = df
        self._grid = grid
        self._active_pos = []
        self._cipher_pos = []
    
    def highlighty(self, df, color='yellow'):
        """ helper method for colouring grid cells in yellow """
        return 'background-color: {}'.format(color)

    def highlightr(self, df, color='#FF0000'):
        """ helper method for colouring grid cells in red """
        return 'background-color: {}; color: #FFFFFF'.format(color)
    
    def highlightb(self, df, color='#0000FF'):
        """ helper method for colouring grid cells in blue """
        return 'background-color: {}; color: #FFFFFF'.format(color)

    def highlightg(self, df, color='#00FF00'):
        """ helper method for colouring grid cells in green """
        return 'background-color: {}'.format(color)

    def apply_grid(self):
        """ Draws a grid on the dataframe and returns the style object """
        return self._df.style.applymap(
            self.highlighty, subset=pd.IndexSlice[:, self._grid[0]]
        ).applymap(
            self.highlighty, subset=pd.IndexSlice[self._grid[1], :]
        ).applymap(
            self.highlighty, subset=pd.IndexSlice[:, self._grid[2]]
        ).applymap(
            self.highlighty, subset=pd.IndexSlice[self._grid[3], :]
        ).set_table_attributes(
            'style="font-size: 10px"'
        )

    def active(self, pos):
        """
        Set a given position active
        
        :param: string pos
        
        This will set a given gridref as being an active cell for the ciphertext,
        colouring it red on the grid
        """
        if pos not in self._active_pos:
            self._active_pos.append(pos)
    
    def lacuna(self, pos):
        """
        marks a given cell as being a 'lacuna' of the current cipher character
        
        :param: string pos
        """
        if pos not in self._active_pos:
            self._active_pos.append(pos)
        
    def cipher(self, pos):
        """
        marks a given cell as containing the current cipher character
        
        :param: string pos
        """
        if pos not in self._cipher_pos:
            self._cipher_pos.append(pos)

    def _active(self, pos):
        {
            'tl': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[1], self._grid[0]]),
            'tr': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[1], self._grid[2]]),
            'bl': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[3], self._grid[0]]),
            'br': lambda m: m.applymap(self.highlightr, subset=pd.IndexSlice[self._grid[3], self._grid[2]]),
        }[pos](self._applied)
        
    def _cipher(self, pos):
        {
            'tl': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[1], self._grid[0]]),
            'tr': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[1], self._grid[2]]),
            'bl': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[3], self._grid[0]]),
            'br': lambda m: m.applymap(self.highlightg, subset=pd.IndexSlice[self._grid[3], self._grid[2]]),
        }[pos](self._applied)
    
    def apply(self):
        """
        Applies the current grid to the dataframe and returns the style object for rendering.
        """
        self._applied = self.apply_grid()
        _ = [self._cipher(pos) for pos in self._cipher_pos]
        _ = [self._active(pos) for pos in self._active_pos]
        return self._applied

It seems to be possible to rotate the order the alphabets are written in and always obtain 1 of the letters from that order... I am not sure if this is a hindrance, a help or the solution yet.

This is something to be very careful with. Whilst in most instances it seems to have minimal impact on the cipher, in some, it changes the shape drastically.

In [5]:
class Table(object):
    """
    The Table class is a wrapper for a pandas DataFrame class with functionality
    to load the table, sort it and create the keys on it.
    """
    table = None
    polarity = ''
    ciphertext = None
    lacuna     = None
    poles      = None
    
    keys = {}
    
    _order = [14, 6, 6, 12,]
    
    def __init__(self, ciphertext, polarity):
        self.ciphertext = ciphertext
        self.lacuna = ''.join([helpers.distancefrom(c, 'Z') for c in ciphertext])
        
        self.polarity = polarity
        self.table = []
        
        self.poles = {
            True: 'E',
            False: 'M',
        }
        
    def create(self):
        """
        Creates A pandas DataFrames from the current instance
        
        This table will either be 13x13 in size if all rows are even, or
        13x26 in size if the rows are a mixture of odd and even characters.
        """
        for distance, pole in helpers.distance_calculator(self.ciphertext, self.lacuna):
            if pole == self.poles[self.polarity]:
                self.table.append([helpers.a2i(c) for c in distance])
        
        self.table = pd.DataFrame(self.table)
        self.table.columns = [helpers.alphabet[(i-1)] for i in self.table.iloc[0]]
        if self.table.shape[0] < 13:
            self.table.loc[len(self.table)] = [
                26 if i % 2 == 0 else 13 for i in self.table.iloc[0]
            ]
        self.table = self.table.loc[
            :,~self.table.columns.duplicated()
        ].sort_values(by=0, axis=1)
        self.table.columns = [i for i in range(1, self.table.shape[1] + 1)]
        self.table.index   = [i for i in range(1, self.table.shape[0] + 1)]
        self.keys = self.create_keys()
        
    def create_keys(self):
        """
        Creates a set of keys for the current table.
        
        This method hard-wires a set of replacement characters which may well belong as a
        definition in the helpers class.
        """
        keys = {
            'replace': {
                'M': 'K', 'V': 'J', 'Z': 'V', 'K': 'V',
            }
        }
        pairings = [
            (helpers.i2a(x), helpers.i2a(x+13)) for x in range(1, 14)
        ]
        
        keys['top']    = helpers.alphabet
        keys['bottom'] = helpers.alphabet[::-1]
        keys['left']   = self.order(self._order[0], pairings)
        keys['right']  = self.order(self._order[1], pairings)
        
        if self.polarity:
            keys['top']    = self.order(self._order[2], pairings)
            keys['bottom'] = self.order(self._order[3], pairings)
        
        return keys
    
    def order(self, start, pairings, axis=1):
        """
        Orders the current keys into a set order
        
        :param: int   start    The index to start the order sequence from
        :param: list  pairings A list of tuples representing (character, (character+13))
        
        :return: list
        """
        keys = []
        increment = start
        for _ in range(self.table.shape[axis]):
            for pair in pairings:
                if helpers.i2a(start) in pair:
                    keys.append(pair)
                    break
            start = (start + increment) % 26
        return keys

    def __getattr__(self, what):
        """
        We pass most calls to the dataframe here.
        """
        try:
            return getattr(self.__class__, what)
        except AttributeError:
            pass
        return getattr(self.table, what)

In [6]:
class Square(object):
    """
    Used to calculate the shape of the grid to plot on the dataframe
    """
    tl            = ''
    bl            = ''
    tr            = ''
    br            = ''
    replace       = None
    character     = ''
    _grid         = []
    _highlight    = None
    lacuna_active = False
    cipher_active = False
    alt_active    = False
    use_alt       = False
    polarity      = False
    table         = None
    ORDER         = [
        'tl', 'tr', 'br', 'bl'
    ]
    
    def __init__(self, character, polarity, use_alt, ciphertext):
        self.character = character
        character_index = helpers.a2i(character)
        self.table = Table(ciphertext, polarity)
        self.use_alt = use_alt
        self.tl = self.bl = self.tr = self.br = ''
 
        
    def plot(self):
        """
        Plot the grid using the current character, the polarity and whether the 
        current character is to be replaced or not
        """
        self.table.create()
        self.replace = self.table.keys['replace']
        
        alt_char = self.replace[self.character] \
            if self.use_alt and self.character in self.replace.keys() \
            else self.character
        
        self.alt_active = alt_char != self.character
        
        top    = [
            i for i in range(
                len(self.table.keys['top'])
            ) if alt_char in self.table.keys['top'][i]
        ][0] + 1
        
        right  = [
            i for i in range(
                len(self.table.keys['right'])
            ) if alt_char in self.table.keys['right'][i]
        ][0] + 1
        
        bottom = [
            i for i in range(
                len(self.table.keys['bottom'])
            ) if self.character in self.table.keys['bottom'][i]
        ][0] + 1
        
        left   = [
            i for i in range(
                len(self.table.keys['left'])
            ) if self.character in self.table.keys['left'][i]
        ][0] + 1
        
        self._grid = [
            top     if top < bottom else bottom,
            left    if left < right else right,
            bottom  if bottom > top else top,
            right   if right > left else left
        ]

        self.tl = self.table.loc[self._grid[1], self._grid[0]]
        self.tr = self.table.loc[self._grid[1], self._grid[2]]
        self.bl = self.table.loc[self._grid[3], self._grid[0]]
        self.br = self.table.loc[self._grid[3], self._grid[2]]
        self._highlight = Highlighter(self.table, self.gridref)
        self.markcipher(self.character)
    
    def get(self):
        """
        Return the current grid, clockwise from top left
        """
        return [self.tl, self.tr, self.br, self.bl]
    
    def markcipher(self, char, recurse=True):
        """
        Marks a given grid square as a cipher characer
        
        :param: char char
        :param: bool recurse
        
        :If recurse is True, will call itself with the inverse position
        """
        inverse = helpers.distancefrom(char, 'Z')
        char = helpers.a2i(char)
        pos = ''
        if char in self.get():
            pos = Square.ORDER[self.get().index(char)]
            self._highlight.cipher(pos)
            
            if recurse:
                self.cipher_active = pos
            else:
                self.lacuna_active = pos
        if recurse:
            self.markcipher(inverse, False)
    
    def active(self, corner):
        """ Wrapper for Highlight.active """
        self._highlight.active(corner)
        return self.active_char(corner)
    
    def active_char(self, pos):
        """
        Find the active character at a given position
        
        :param: string pos
        
        :return: int
        """
        return {
            'tl': self.table.loc[self._grid[1], self._grid[0]],
            'tr': self.table.loc[self._grid[1], self._grid[2]],
            'bl': self.table.loc[self._grid[3], self._grid[0]],
            'br': self.table.loc[self._grid[3], self._grid[2]],
        }[pos]
        
    def contains(self, what):
        """ Test if a given character exists here """
        return helpers.a2i(what) in self.get()
    
    def mark_lacuna(self, character):
        """ Wrapper for Highlight.lacuna """
        position = helpers.distancefrom(character, 'Z')
        if self.contains(position):
            position = Square.ORDER[self.get().index(helpers.a2i(position))]
            self._highlight.lacuna(position)
    
    @property
    def gridref(self):
        return self._grid
    
    @property
    def apply(self):
        return self._highlight.apply()

In [7]:
51 // 26

1

In [8]:
globalout = Output()
class Cipher(object):
    """
    Main cipher class
    
    Creates a list Character objects used to map the cipher into plaintext
    """
    cipher    = None
    alphabet  = None
    _vbox     = None
    _hbox     = None
    _label    = None
    _event    = None
    _html     = None
    _use      = 'cipher'
    _cindex   = 0
    _currentr = 0 
    _currentc = 0

    def __init__(self, ciphertext, invert=False):
        self.cipher = []
        self.ciphertext = ciphertext.upper()
        
        # ------------------------------------------------------------
        # `lacunatext` is the full ciphertext, each character removed
        # from Z.
        # ------------------------------------------------------------
        self.lacunatext = ''.join(
            [helpers.distancefrom(c, 'Z') for c in self.ciphertext]
        )
        if invert:
            self.ciphertext = self.lacunatext
            self.lacunatext = ciphertext
        
        helpers.distance_calculator(ciphertext, self.lacunatext)
        
        # ------------------------------------------------------------
        # A polarity table is formed. This is used to help determine
        # the rules. As each character is found, the polarity of that
        # character changes
        # ------------------------------------------------------------
        self.alphabet = {
            character: False for character in helpers.alphabet
        }
        
        # ------------------------------------------------------------
        # Create an object for each character setting the value of
        # 'use_alt' to the current value of the boolean alphabet.
        # We then  invert the alphabet flag for the next occurance of
        # that character.
        # ------------------------------------------------------------
        for i, c, l in zip(range(1, self.length + 1), self.ciphertext, self.lacunatext):
            cipher_flag = self.alphabet[c] if c not in ['M', 'Z'] else True
            lacuna_flag = self.alphabet[c] if l not in ['M', 'Z'] else True
            self.cipher.append(
                Character(c, i, cipher_flag, ciphertext)
            )
            
            self.alphabet[c] = not self.alphabet[c]
        
        # Used for displaying the current position on the ciphergrid
        self._currentc = 'A'
        self._currentr = 0

    def setup_jupyter(self):
        """
        Sets up elements on the page for use with a Jupyter notebook.
        """
        self._label = Label('Move the cursor over the cell and use the left and right arrow keys to navigate')
        self._hbox = HBox()
        self._html = HTML('<h3>Label position?</h3>')
        
        self._inner = VBox()
        self._vbox = VBox([self._html, self._inner, self._label])
        self._event = Event(source=self._vbox, watched_events=['keydown'])
        self._event.on_dom_event(self.handle_event)

    @property
    def length(self):
        """ Return the length of the current cipher """
        return len(self.ciphertext)

    def intermediate(self, pos):
        """ Get the intermediate character from the cipher """
        return self[pos].decipher
    
    def __str__(self):
        return ''.join([str(c) for c in self])
    
    def __iter__(self):
        return getattr(self, self._use).__iter__()
    
    def __getitem__(self, key):
        return getattr(self, self._use).__getitem__(key)
    
    def __len__(self):
        return self.length
    
    @globalout.capture()
    def handle_event(self, event):
        """ Jupyter ipyevents binding code """
        if 'code' in event.keys():
            self.setposition(event['code'])
            self._draw()
            
    def setposition(self, code):
        """
        Sets the current cipher position on the grids
        """
        pagesize = 5
        if code == 'ArrowLeft':
            self._cindex = self._cindex - 1 if self._cindex > 0 else len(ciphertext)-1
        elif code == 'ArrowRight':
            self._cindex = self._cindex + 1 if self._cindex < len(ciphertext)-1 else 0
        elif code == 'ArrowUp':
            self._cindex = self._cindex + pagesize if (self._cindex + pagesize) < len(ciphertext)-1 \
                else 0 + ((self._cindex + pagesize) - len(ciphertext))
        elif code == 'ArrowDown':
            self._cindex = self._cindex - pagesize if (self._cindex - pagesize) >= 1 \
                else (len(ciphertext) - (pagesize - self._cindex))

        self._currentc = helpers.i2a((self._cindex % 26) + 1)
        self._currentr = self._cindex // 26
        
    
    def _draw(self):
        """ Jupyter notebook code to draw widgets """
        left       = Output()
        right      = Output()
        properties = Output()
        decipher   = Output()
        deciphered = Output()
        
        tables = {
            True:  [],
            False: []
        }
        for key in self[self._cindex].cipher.keys():
            characters = self[self._cindex].cipher[key].get()
            for i, character in zip(range(len(characters)), characters):
                partial = Output()
                df = self[self._cindex].all_positions(helpers.i2a(character))
                df = df.style.set_caption(
                    '{} ({})'.format(character, helpers.i2a(character))
                ).set_table_attributes(
                    'style="font-size: 10px"'
                )
                with partial:
                    display.display(df)
                tables[key].append(partial)
        
        with left:
            display.display(self[self._cindex].cipher[True].apply)
        with right:
            display.display(self[self._cindex].cipher[False].apply)
        
        with properties:
            display.display(
                self[self._cindex].condition_table
                    .style.set_caption('Properties')
                    .set_table_attributes(
                        'style="font-size: 10px"'
                    )
            )
            
        with decipher:
            display.display(
                self[self._cindex]
                    .all_positions()
                    .style.set_caption('Selected')
                    .set_table_attributes(
                        'style="font-size: 10px"'
                    )
            )
        
        with deciphered:
            display.display(self.as_dataframe())
        
        subtables = VBox()
        subtables.children = [HBox(tables[True]), HBox(tables[False])]
        
        self._hbox.children = [left, right]
        self._inner.children = [
            self._hbox,
            HBox([properties, decipher, subtables, deciphered]),
            globalout
        ]
        self._html.value = '<h3>Current character {}, index {}, deciphered to {}</h3>'.format(
            self[self._cindex].character,
            self._cindex + 1,
            str(self[self._cindex])
        )

    def as_dataframe(self, ciphertext=False):
        """
        display the finished cipher in a dataframe
        """
        n = 26
        ciphertext = [str(i) for i in self] if not ciphertext else [i for i in ciphertext]
        df = pd.DataFrame(
            [self[i:i + n] for i in range(0, len(ciphertext), n)]
        )
        mask = df.applymap(lambda x: x is None)
        cols = df.columns[(mask).any()]
        for col in df[cols]:
            df.loc[mask[col], col] = ''
        df.columns = helpers.alphabet
        return df.style.hide_index().set_caption(
            'Deciphered plaintext'
        ).set_table_attributes(
            'style="font-size: 10px"'
        ).applymap(
            Highlighter(None, None).highlightg,
            subset=pd.IndexSlice[self._currentr, self._currentc]
        )
        
    def display(self, index=1):
        """
        Display a given character in a Jupyter cell
        """
        index = 1 if index == 0 else index
        display.clear_output(wait=True)
        if index is not None:
            self._cindex = (index - 1)
            self._currentc = helpers.i2a((self._cindex % 26)+1)
        self._currentr = self._cindex // 26
        self._draw()
        display.display(self._vbox)

In [9]:
class Character(object):
    """
    The outer core class of the cipher
    
    If Helpers represents the core of the cipher, this is its counterpart
    
    The Character class takes a given input character and associated index, then maps it
    to one of 8 possible grid references, then from there chooses a single calculation
    method to turn that grid square into a potential plaintext character.
    
    It does this by following a complex set of rules defined in the `decipher` and `unpack_active`
    methods
    
    The `decipher` method defines 4 primary rules:
    
    - neither cipher or lacuna visible
    - cipher visible
    - lacuna visible
    - both visible
    
    If neither cipher or lacuna are visible in the grids plotted to either table, the decipher method
    then defines an additional set of rules for choosing the correct cipher algorithm.
    
    If the inverse is true and one or both of the cipher/lacuna is visible, we then use the `unpack_active` method
    
    The `unpack_active` method takes the current square only if it has a ciphertext character
    and/or a lacuna text character visible in the same grid, then attempts to map the ciphertext
    character onto one of 4 possible deciphering algorithms.
    
    They both work on similar principles. First choosing the grid square, then applying rules to
    the value found in that square along with any boolean values generated from:
    
    - The polarity of the current character (is it an odd or even character)
    - an alternating binary flip-switch based on whether that cell was previously active.
    - Mod 2, 5, 15 on the current index
    """
    cipher = {
        True: None,
        False: None,
    }

    index              = 0
    character          = ''
    deciphered         = ''
    table              = None
    xor                = None
    position           = None
    use_alt            = False
    cipher_active      = False
    lacuna_active      = False
    map_cipher         = {}
    _intermediate      = None
    _calculation_index = 0

    def __init__(self, character, index, use_alt, ciphertext):
        self.index     = index
        self.character = character.upper()
        self.use_alt   = use_alt

        # ------------------------------------------------------------
        # Create a Square object for each character in the cipher
        # ------------------------------------------------------------
        self.cipher = {
            key: Square(self.character, key, self.use_alt, ciphertext)
            for key in self.cipher.keys()
        }

        # ------------------------------------------------------------
        # We set the current polarity against the cipher character
        # polarity, then build the tables and move to find the
        # intermediate character.
        # ------------------------------------------------------------
        self.xor = helpers.a2i(self.character) % 2 == 0
        _ = [table.plot() for _, table in self.cipher.items()]
        _ = self.decipher

    def __str__(self):
        return self.final[1]

    def __repr__(self):
        return str(self)

    @property
    def condition_table(self):
        """
        Return the properties of the current object as a pandas DataFrame
        """
        table = {
            'table': self.table,
            'xor': self.xor,
            'position': self.position,
            'use_alt': self.use_alt,
            'uses_alt': self.uses_alt,
            'cipher_active': self.cipher_active,
            'lacuna_active': self.lacuna_active,
        }
        return pd.DataFrame(list(table.items()), columns=['Property', 'Value'])

    @property
    def decipher(self):
        """
        Main decipher method
        """
        if not self._intermediate:
            self.cipher_active = self.cipher[True].cipher_active \
                if self.cipher[True].cipher_active else self.cipher[False].cipher_active

            self.lacuna_active = self.cipher[True].lacuna_active \
                if self.cipher[True].lacuna_active else self.cipher[False].lacuna_active

            # ============================================================================
            # RULES
            # ----------------------------------------------------------------------------
            # The following block sets the rules for the cipher location starting with the
            # principle conditions for execution.
            #
            # Deciphering starts in the top left corner and moves around the table according
            # to the rules matched for that character.
            # ============================================================================
            self.position = 'tl'
            self.table = (self.index % 2 != 0)
            # ------------------------------------------------------------
            # _calculation_index is used to select which deciphering
            #                    algorithm to choose from 4 possible
            #                    options.
            #
            # The order of the calculation index is fixed as this maps
            # directly to a list of  deciphering algorithms below.
            #
            # The order is as follows:
            #   - 0: Nothing
            #   - 1: (c + x) % 26
            #   - 2: distancefrom(x, 'Z')
            #   - 3: c + distancefrom(x, 'Z') % 26
            # ------------------------------------------------------------
            self._calculation_index = 1 if not self.table or (self.table and self.xor) else 0
            self._calculation_index = 0 if self.uses_alt else self._calculation_index
            self._calculation_index += 2 if self.can_replace(self.character) and not self.uses_alt else 0
            self._calculation_index += 1 if self.index % 15 == 0 else 0
            self._calculation_index += 1 if helpers.a2i(self.character) % 5 == 0 else 0

            self.table = not self.table if (helpers.a2i(self.character) % 2) == 0 else self.table
            
            """
            Condition 1. If we use the alternate character, change the nature of the table
            """
            if self.use_alt and self.uses_alt:
                self.table = not self.table if self.index % 2 == 0 else self.table
                self.position = 'tr'
                self.table, self.position = (not self.table, 'tl') if self.xor else (self.table, self.position)

            """
            Condition 2. If the current polarity of the character is True, flip positions
            """
            if self.xor:
                self.position = {
                    'tl': 'br',
                    'br': 'tl',
                    'tr': 'bl',
                    'bl': 'tr',
                }[self.position]

            """
            Primary Rule 1. If not cipher character visible and not lacuna character visible
            """
            if all([not self.cipher_active, not self.lacuna_active]):
                # ------------------------------------------------------------
                # Sub rule 1 - Current characters Z Lacuna exists in either table
                # ------------------------------------------------------------
                c = helpers.a2i(self.character)
                l = helpers.a2i(helpers.distancefrom(self.character, 'Z'))
                c = (c + c) % 26
                l = (l + l) % 26
                pos = False
                if l in self.cipher[False].get():
                    pos = self.cipher[False].get().index(l)
                    if c in self.cipher[True].get():
                        pos = self.cipher[True].get().index(c)

                elif l in self.cipher[True].get():
                    pos = self.cipher[True].get().index(l)
                    if c in self.cipher[False].get():
                        pos = self.cipher[False].get().index(c)

                if pos:
                    self.table = not self.table
                    self.position = Square.ORDER[pos]

                # ------------------------------------------------------------
                # Go back to top left if we're XOR and in the mixed character table
                # then set a new calculation if we're not mod 5 (inverse 5 minute rule)
                # ------------------------------------------------------------
                if not self.table and self.xor:
                    self.position = 'tl'
                    self._calculation_index += 0 if helpers.a2i(self.character) % 5 == 0 else 1
                
                # ------------------------------------------------------------
                # Map current position on to deciphering algorithm index
                # ------------------------------------------------------------
                five_minute = self.index % 5 == 0 and self.table
                tl_direction = self.table and not self.xor and not self.use_alt
                self._calculation_index += {
                    'tl': 2 if tl_direction and not five_minute else 0,
                    'br': 0 if self.table else 0,
                    'tr': 0 if self.table else 0,
                    'bl': 0 if self.table else 0,
                    False: 0,
                }[self.position]
            elif self.cipher_active and not self.lacuna_active:
                """
                Primary Rule 2. If cipher character is visible and lacuna character is not visible
                """
                self.position = self.unpack_active(
                    self.cipher[True].cipher_active,
                    self.cipher[False].cipher_active,
                )
                if self.index == 7:
                    print('{} {}'.format(self.character, self.position))
            elif self.lacuna_active and not self.cipher_active:
                """
                Primary Rule 3. If lacuna character is visible and cipher character is not visible
                """
                self.position = self.unpack_active(
                    self.cipher[True].lacuna_active,
                    self.cipher[False].lacuna_active,
                    True
                )
            elif self.cipher_active and self.lacuna_active:
                """
                Primary Rule 4. If cipher character is visible and lacuna character is not visible
                """
                a = self.unpack_active(
                    self.cipher[True].cipher_active,
                    self.cipher[False].cipher_active,
                )
                b = self.unpack_active(
                    self.cipher[True].lacuna_active,
                    self.cipher[False].lacuna_active,
                    True
                )
                self.position = 'tl' if a == b else 'br'

            # ============================================================================
            # END RULES
            # ============================================================================
            self._intermediate = helpers.i2a(self.cipher[self.table].active(self.position))
            for key in self.cipher.keys():
                self.cipher[key].mark_lacuna(self._intermediate)
        return self._intermediate

    @property
    def uses_alt(self):
        return all([self.cipher[True].alt_active, self.cipher[False].alt_active])

    def can_replace(self, what):
        return what in self.cipher[True].table.keys['replace'].keys()

    def unpack_active(self, even_active, mixed_active, lacuna=False):
        """
        Used to determine the additional rules surrounding any combination
        of cipher and lacuna visibility in the tables.
        
        :param: string|bool even_active   if not False, represents the visibility of the cipher
                                          or lacuna character in the even numbered table
        :param: string|bool lacuna:active if not False, represents the visibility of the cipher
                                          or lacuna character in the mixed polarity table

        :return: string
        
        The presence of one or both of these characters can drastically alter the result of the
        final cipher.
        
        This will return one of
        
        - `tl` Top left
        - `tr` Top right
        - `br` Bottom right
        - `bl` Bottom left
        """
        self.table = not self.table if helpers.a2i(self.character) % 2 != 0 else self.table

        if not self.uses_alt:
            self.table = not self.table if self.lacuna_active \
                and (even_active and not mixed_active) \
                else self.table

        # ============================================================================
        # SECONDARY RULES
        # ============================================================================
        order_table_even = {
            'tl': 0 if lacuna else 1,
            'br': 2,
            'tr': 3 if lacuna and (self.index % 2) != 0 else 1,
            #'bl': 2 if lacuna else 1, # think this one
            'bl': 2 if lacuna and (self.index % 2) != 0 \
                else 0 if not lacuna and (self.xor and not self.use_alt) \
                else 1,
            False: 0,
        }[even_active]

        order_table_mixed = {
            'tl': 0,
            'br': 2,
            'tr': 2 if self.table and not self.use_alt else \
                0 if not lacuna else \
                1 if self.index % 5 == 0 else 3,
            'bl': 3 if self.table or not (self.table and lacuna) else 2,
            False: 0,
        }[mixed_active]

        self._calculation_index += sum([order_table_even, order_table_mixed]) % 4
        validate = even_active if not mixed_active else mixed_active
        
        if even_active and mixed_active:
            validate = even_active + mixed_active
            if validate in ['bltl',]:
                self.table = not self.table
            return {
                'tlbr': 'tl',
                'trbr': 'bl',
                'brtr': 'tl',
                'bltl': 'tr',
            }[validate]
        
        # 5 minute rule
        if self.index % 2 != 0 and helpers.a2i(self.character) % 5 == 0:
            validate = 'br' if even_active and not mixed_active else validate

        even = even_active and not mixed_active
        even_table = even and self.table
        mixed_table = not even and self.table

        return {
            'tl': 'bl',
            'tr': 'bl',
             # added xor for c7
            'bl': 'br' if (self.table and not self.uses_alt) else \
                'bl' if not self.use_alt and self.xor else 'tl',
            'br': 'tr',
        }[
            validate
            if not self.uses_alt or not self.xor else self.position
        ]

    @property
    def final(self):
        self._calculation_index = self._calculation_index % 4
        return (
            self._calculation_index,
            self.transcribe(self._calculation_index, self.decipher)
        )

    def transcribe(self, position, character):
        """
        For a given table index, translate the position to the one directly between ciphertext and plaintext

        :param int: position

        :return: character

        Given ciphers as:

               QQPRNGKSSNYPVTTMZFPK     IIJHLSOGGLAJDFFMZTJO
            ------------------------------------------------
            1. DCBVPGCUTVWBVCXWNYQS  2. VWXDJSWEFDCXDWBCLAIG
            3. UTRNDNNNMJVRRWRJNEGD  4. EFHLVLLLMPDHHCHPLUSV

        If the deciphered character is in position 1, we add this to the cipher character to obtain cipher 3
        If cipher character is in cipher 2, we first subtract this from Z to obtain cipher 1, then add
        If cipher character is in cipher 3, we do nothing
        If cipher character is in cipher 4, we subtract this from Z.
        
        Ciphers are listed in the order 3, 1, 4, 2
        """
        return helpers.distancefrom(
            self.character,[
                lambda x, _: x,
                lambda x, c: helpers.i2a(
                    (helpers.a2i(c) + helpers.a2i(x)) % 26
                ),
                lambda x, _: helpers.distancefrom(x, 'Z'),
                lambda x, c: helpers.i2a(
                    (helpers.a2i(c) + helpers.a2i(helpers.distancefrom(x, 'Z'))) % 26
                ),
            ][position](character, self.character)
        )

    def all_positions(self, character=None):
        """
        Get a dataframe containing all characters reachable from a given reference
        
        :param: char character
        :return: pandas.DataFrame
        
        If character is None, we use the current decipher character
        """
        if not character:
            character = self.decipher
        return pd.DataFrame(
            [[
                self.transcribe(0, character),
                self.transcribe(1, character),
            ],[
                self.transcribe(3, character),
                self.transcribe(2, character),
            ]]
        )


In [10]:
%%time
cipher = Cipher(ciphertext)

X bl
CPU times: user 6.48 s, sys: 30.8 ms, total: 6.51 s
Wall time: 6.52 s


When activated, the cell below will display:

- Two primary tables, the evens table (all numbers even) and the mixed table.
  - Each table will have a grid laid out on it in yellow
    - red block is the cell selected for the cipher character
    - green is the cipher and lacuna characteers
- A properties table. This is all the flags which make up the rules
- The values reachable from the selected cipher cell
- The remaining selected cells read clockwise from top left

## Warning.

As you navigate through, you may think you see words appearing.
Do not assume that just because you might be able to make a word, that it is the correct one.
There are a total of 32 characters reachable from any single ciphertext character. Let the 
rules guide you and try not to 'invent' rules just because you "want" a particular character
to appear.

Also just because there are already words populated, this does not make them right. Not all
rules have been completed. The only words that are accurate are NORTHEAST, BERLIN and CLOCK
as these come directly from Mr Sandborn.

> Any existing rules written are not necessarily correct at this stage. Do not be afraid to
> change them if you think there is a better rule.

The rules, once complete will be logical and probably straightforward and although they may
seem complicated at this stage. I will virtually guarantee that they will not appear so by the time this method is complete.

The rules must be understood before this cipher is complete. Only then will the true message appear.

The numbering system of this table starts at 1. This is deliberate and should not be changed as it will
change the nature of the entire cipher from this point. You can set this to start at any position by
setting the index to `display`. For example `display(64)` will move to the 64th character (N from NYPVTT)

Use the cursor keys to navigate the table.

- Right arrow: Move forwards 1 character. Loops back to the start of the cipher
- Left arrow: Move backwards 1 character. Loops back to the end of the cipher
- Up arrow: Move forwards 5 characters
- Down arrow: Move backwards 5 characters

In [11]:
cipher.setup_jupyter()
cipher.display()